In [1]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
from scipy.io import arff
import pandas as pd

In [2]:
# load data
df=pd.read_csv('InternetAds_norm_19.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,att4,att5,att6,att7,att8,att9,att10,att11,att12,att13,...,att1550,att1551,att1552,att1553,att1554,att1555,att1556,att1557,att1558,outlier
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [3]:
df.shape

(3264, 1556)

In [4]:
# data to values
df = df.values

In [5]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/InternetAds.json")
gc = GCForest(config)

In [15]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,1555]
X = df[:,0:1555]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
# number of class
len(np.unique(y_train))

2

In [8]:
          # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-04-25 00:31:32,182][cascade_classifier.fit_transform] X_groups_train.shape=[(2186, 1555)],y_train.shape=(2186,),X_groups_test.shape=[(1078, 1555)],y_test.shape=(1078,)
[ 2018-04-25 00:31:32,199][cascade_classifier.fit_transform] group_dims=[1555]
[ 2018-04-25 00:31:32,201][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 00:31:32,202][cascade_classifier.fit_transform] group_ends=[1555]
[ 2018-04-25 00:31:32,204][cascade_classifier.fit_transform] X_train.shape=(2186, 1555),X_test.shape=(1078, 1555)
[ 2018-04-25 00:31:32,222][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2186, 1555), X_cur_test.shape=(1078, 1555)
[ 2018-04-25 00:31:33,000][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.26%
[ 2018-04-25 00:31:33,990][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=96.35%
[ 2018-04-25 00:31:34,886][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

In [9]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-25 00:31:40,811][cascade_classifier.transform] X_groups_test.shape=[(1078, 1555)]
[ 2018-04-25 00:31:40,816][cascade_classifier.transform] group_dims=[1555]
[ 2018-04-25 00:31:40,817][cascade_classifier.transform] X_test.shape=(1078, 1555)
[ 2018-04-25 00:31:40,821][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1078, 1555)


Test Accuracy of ExtraTrees = 97.031540 %


In [10]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-25 00:31:42,353][cascade_classifier.transform] X_groups_test.shape=[(1078, 1555)]
[ 2018-04-25 00:31:42,357][cascade_classifier.transform] group_dims=[1555]
[ 2018-04-25 00:31:42,359][cascade_classifier.transform] X_test.shape=(1078, 1555)
[ 2018-04-25 00:31:42,364][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1078, 1555)


In [11]:
# New data
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-25 00:31:44,084][cascade_classifier.fit_transform] X_groups_train.shape=[(2186, 1557)],y_train.shape=(2186,),X_groups_test.shape=[(1078, 1557)],y_test.shape=(1078,)
[ 2018-04-25 00:31:44,107][cascade_classifier.fit_transform] group_dims=[1557]
[ 2018-04-25 00:31:44,109][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 00:31:44,111][cascade_classifier.fit_transform] group_ends=[1557]
[ 2018-04-25 00:31:44,113][cascade_classifier.fit_transform] X_train.shape=(2186, 1557),X_test.shape=(1078, 1557)
[ 2018-04-25 00:31:44,134][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2186, 1557), X_cur_test.shape=(1078, 1557)
[ 2018-04-25 00:31:44,745][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.72%
[ 2018-04-25 00:31:45,499][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=96.80%
[ 2018-04-25 00:31:46,248][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

In [12]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-25 00:31:52,122][cascade_classifier.transform] X_groups_test.shape=[(1078, 1557)]
[ 2018-04-25 00:31:52,130][cascade_classifier.transform] group_dims=[1557]
[ 2018-04-25 00:31:52,132][cascade_classifier.transform] X_test.shape=(1078, 1557)
[ 2018-04-25 00:31:52,139][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(1078, 1557)


Test Accuracy of ExtraTrees = 96.567718 %


In [13]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')

In [14]:
# Deep
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()

[ 2018-04-25 00:31:53,740][cascade_classifier.fit_transform] X_groups_train.shape=[(2186, 1557)],y_train.shape=(2186,),X_groups_test.shape=[(1078, 1557)],y_test.shape=(1078,)
[ 2018-04-25 00:31:53,756][cascade_classifier.fit_transform] group_dims=[1557]
[ 2018-04-25 00:31:53,757][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-25 00:31:53,759][cascade_classifier.fit_transform] group_ends=[1557]
[ 2018-04-25 00:31:53,760][cascade_classifier.fit_transform] X_train.shape=(2186, 1557),X_test.shape=(1078, 1557)
[ 2018-04-25 00:31:53,778][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(2186, 1557), X_cur_test.shape=(1078, 1557)
[ 2018-04-25 00:31:54,496][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.26%
[ 2018-04-25 00:31:55,560][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=96.80%
[ 2018-04-25 00:31:56,330][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

('Test Accuracy of ExtraTrees = 96.846011 %', 0, 'fois')


[ 2018-04-25 00:32:04,764][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.26%
[ 2018-04-25 00:32:05,690][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=96.80%
[ 2018-04-25 00:32:06,500][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=95.89%
[ 2018-04-25 00:32:07,283][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.72%
[ 2018-04-25 00:32:08,191][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.72%
[ 2018-04-25 00:32:08,990][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.17%
[ 2018-04-25 00:32:09,762][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=96.35%
[ 2018-04-25 00:32:10,743][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=96.79%


('Test Accuracy of ExtraTrees = 95.732839 %', 1, 'fois')


[ 2018-04-25 00:32:14,947][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.26%
[ 2018-04-25 00:32:15,688][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=96.80%
[ 2018-04-25 00:32:16,563][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=96.35%
[ 2018-04-25 00:32:17,354][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.17%
[ 2018-04-25 00:32:18,135][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.72%
[ 2018-04-25 00:32:18,898][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.63%
[ 2018-04-25 00:32:19,644][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.26%
[ 2018-04-25 00:32:20,548][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.71%


...

('Test Accuracy of ExtraTrees = 96.938776 %', 17, 'fois')


[ 2018-04-25 00:34:49,962][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.26%
[ 2018-04-25 00:34:50,886][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.72%
[ 2018-04-25 00:34:51,856][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=96.35%
[ 2018-04-25 00:34:52,736][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.72%
[ 2018-04-25 00:34:53,653][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.26%
[ 2018-04-25 00:34:54,542][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.17%
[ 2018-04-25 00:34:55,307][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=97.26%
[ 2018-04-25 00:34:56,110][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.25%


('Test Accuracy of ExtraTrees = 96.753247 %', 18, 'fois')


[ 2018-04-25 00:35:00,553][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=97.26%
[ 2018-04-25 00:35:01,454][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=97.26%
[ 2018-04-25 00:35:02,207][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=96.80%
[ 2018-04-25 00:35:03,004][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=97.72%
[ 2018-04-25 00:35:03,972][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=97.26%
[ 2018-04-25 00:35:04,751][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.63%
[ 2018-04-25 00:35:05,688][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=96.35%
[ 2018-04-25 00:35:06,486][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=97.25%


('Test Accuracy of ExtraTrees = 96.753247 %', 19, 'fois')


# ('Test Accuracy of ExtraTrees = 97.124304 %', 3, 'fois')